#### IMPORT STATEMENTS

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

#### LOADING DATA

In [ ]:
# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Defining transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Loading CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Defining data loaders
batch_size = 128
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:01<00:00, 94015168.88it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#### CNN ARCHITECTURES

In [ ]:


# CNN1 Architecture
cnn1 = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(64 * 8 * 8, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)


# CNN2 Architecture
cnn2 = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(64 * 8 * 8, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

# CNN3 Architecture
cnn3 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128 * 8 * 8, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

# CNN4 Architecture
cnn4 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(256 * 4 * 4, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
)

# CNN5 Architecture
cnn5 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Dropout(0.25),
    nn.Flatten(),
    nn.Linear(256 * 4 * 4, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)



#### Defining loss function and optimizer

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(cnn1.parameters(), lr=0.001)
optimizer2 = optim.Adam(cnn2.parameters(), lr=0.001)
optimizer3 = optim.Adam(cnn3.parameters(), lr=0.001)
optimizer4 = optim.Adam(cnn4.parameters(), lr=0.001)
optimizer5 = optim.Adam(cnn5.parameters(), lr=0.001)

#### TRAINING MODEL AND EVALUATING MODEL

In [ ]:
# Define the training function
def train(model, criterion, optimizer, trainloader, device):
    model.train()
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(trainloader)
def evaluate(model, criterion, testloader, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy, test_loss


In [ ]:
## Set the random seed for reproducibility
torch.manual_seed(42)

# Initialize and train the models
models = [cnn1, cnn2, cnn3, cnn4, cnn5]
num_epochs = 10

for model in models:
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"Training {model.__class__.__name__}...")
    for epoch in range(num_epochs):
        train_loss = train(model, criterion, optimizer, trainloader, device)
        accuracy, test_loss = evaluate(model, criterion, testloader, device)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
        print(f"Accuracy: {accuracy:.2f}")
        print()

    print(f"Finished training {model.__class__.__name__}")



Training Sequential...
Epoch 1/10:
Train Loss: 1.4319 | Test Loss: 93.3486
Accuracy: 0.58

Epoch 2/10:
Train Loss: 1.0611 | Test Loss: 77.9093
Accuracy: 0.65

Epoch 3/10:
Train Loss: 0.8919 | Test Loss: 71.2910
Accuracy: 0.68

Epoch 4/10:
Train Loss: 0.7891 | Test Loss: 68.3596
Accuracy: 0.70

Epoch 5/10:
Train Loss: 0.7046 | Test Loss: 66.0510
Accuracy: 0.71

Epoch 6/10:
Train Loss: 0.6315 | Test Loss: 66.4158
Accuracy: 0.71

Epoch 7/10:
Train Loss: 0.5631 | Test Loss: 68.1223
Accuracy: 0.71

Epoch 8/10:
Train Loss: 0.4917 | Test Loss: 66.4744
Accuracy: 0.72

Epoch 9/10:
Train Loss: 0.4281 | Test Loss: 70.1221
Accuracy: 0.72

Epoch 10/10:
Train Loss: 0.3691 | Test Loss: 72.7444
Accuracy: 0.72

Finished training Sequential
Training Sequential...
Epoch 1/10:
Train Loss: 1.4301 | Test Loss: 92.2501
Accuracy: 0.58

Epoch 2/10:
Train Loss: 1.0648 | Test Loss: 79.7113
Accuracy: 0.64

Epoch 3/10:
Train Loss: 0.9066 | Test Loss: 72.4286
Accuracy: 0.68

Epoch 4/10:
Train Loss: 0.7930 | Test Lo

In [19]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Model", "Parameters", "Accuracy"]
model_names = ["CNN1", "CNN2", "CNN3", "CNN4", "CNN5"]
model_architectures = [cnn1, cnn2, cnn3, cnn4, cnn5]

# Train and evaluate each model
for model_name, model_architecture in zip(model_names, model_architectures):
    # Move the model to the device
    model = model_architecture.to(device)

    # Count the number of parameters
    num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    # Evaluate the model
    accuracy, _ = evaluate(model, criterion, testloader, device)

    # Add a row to the table
    table.add_row([model_name, num_parameters, accuracy])

# Print the comparison table
print(table)

+-------+------------+----------+
| Model | Parameters | Accuracy |
+-------+------------+----------+
|  CNN1 |   545098   |  0.7236  |
|  CNN2 |   545098   |  0.7121  |
|  CNN3 |  2360138   |  0.7541  |
|  CNN4 |  3248202   |  0.7911  |
|  CNN5 |  4216522   |  0.699   |
+-------+------------+----------+
